<a href="https://colab.research.google.com/github/tramlam-ng/scraping_data_tiki_selenium/blob/main/TIKI_Scraper_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install resources

In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 904 kB 9.4 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelea

In [ ]:
### IMPORTS ###
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

#Configuration for Driver and links

In [ ]:
###############
### GLOBALS ###
###############

# Header for chromedriver
HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
# Urls
TIKI                = 'https://tiki.vn'
MAIN_CATEGORIES = [
    {'Name': 'Điện Thoại - Máy Tính Bảng',
     'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Tử - Điện Lạnh',
     'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'},

    {'Name': 'Phụ Kiện - Thiết Bị Số', 
     'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},

    {'Name': 'Laptop - Thiết bị IT', 
     'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},

    {'Name': 'Máy Ảnh - Quay Phim', 
     'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Gia Dụng', 
     'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'},

    {'Name': 'Nhà Cửa Đời Sống', 
     'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 
     'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'},

    {'Name': 'Đồ chơi, Mẹ & Bé', 
     'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'},

    {'Name': 'Làm Đẹp - Sức Khỏe', 
     'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},

    {'Name': 'Thể Thao - Dã Ngoại', 
     'URL': 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner'},

    {'Name': 'Xe Máy, Ô tô, Xe Đạp', 
     'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng quốc tế', 
     'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'},

    {'Name': 'Sách, VPP & Quà Tặng', 
     'URL': 'https://tiki.vn/nha-sach-tiki/c8322?src=c.8322.hamburger_menu_fly_out_banner'},

    {'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 
     'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}
]

# Global driver to use throughout the script
DRIVER = None

#Function to Start and Close Driver

In [ ]:
# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if DRIVER is not None:
        if force_restart:
            DRIVER.close()
        else:
            raise RuntimeError('ERROR: cannot overwrite an active driver. Please close the driver before restarting.')
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

In [ ]:
close_driver()

In [ ]:
start_driver()

In [ ]:
DRIVER.get(TIKI)

In [ ]:
# get links by product types
data = []
for item in MAIN_CATEGORIES:
  url = (item.get('URL'))
  data.append(url)

#Function to scrape info of all products from a Page URL

In [ ]:
# creat function to get link of product
products_all = DRIVER.find_elements_by_class_name('product-item')
def get_link_product(page_url):
  for product in products_all:
    link_product = product.get_attribute('href')
  return link_product

In [ ]:
# get price of product
def get_price_product(page_url):
  price_all = DRIVER.find_elements_by_class_name('price-discount__price')
  for price in price_all:
    price = price.text
  return price

In [ ]:
# get name of product
def get_name_product(page_url):
  name_all = DRIVER.find_elements_by_class_name('name') # name_element is a list
  for name in name_all:
    name = name.find_element_by_tag_name('span').text 

  return name


In [ ]:


# get image of product
def get_img_product(page_url):
  thumbnail_all = DRIVER.find_elements_by_class_name('thumbnail')
  for thumbnail in thumbnail_all:
    image = ''
    try:
      temp = thumbnail.find_elements_by_tag_name('img')
      tmp = temp[1]
      img = tmp.get_attribute('src')
    except Exception as e:
      return ('Cannot find any image')

  return image

In [ ]:
# Function to scrape all products from a page
def get_product_info_from_page(page_url):
    """ Extract info from all products of a specfic page_url on Tiki website
        Args:
            page_url: (string) url of the page to scrape
        Returns:
            data: list of dictionary of products info. If no products shown, return empty list.
    """
    global DRIVER

    data = []
    DRIVER.get(page_url) # Use the driver to get info from the product page
    time.sleep(5) ## Must have the sleep function

    products_all = DRIVER.find_elements_by_class_name('product-item')
    print(f'Found {len(products_all)} products')
    
    price_all = DRIVER.find_elements_by_class_name('price-discount__price')
    name_all = DRIVER.find_elements_by_class_name('name')
    thumbnail_all = DRIVER.find_elements_by_class_name('thumbnail')
    
    for product in products_all:
      #Look through the product and get the data
      # YOUR CODE HERE
      name = get_name_product(page_url)
      price = get_price_product(page_url)
      product_url = get_link_product(page_url)
      image = get_img_product(page_url)

      temp = [name, price, product_url, image]
      data.append(temp)

    return data

#Function to get info from one product

In [ ]:
#################
### FUNCTIONS ###
#################


# Function to extract product info from the product
def get_product_info_single(product_item):
    d = {'name':'',
         'price':'',
         'product_url':'',
         'image':''}

    # name get name through find_element_by_class_name
    try:
        d['name'] = DRIVER.find_elements_by_class_name('name')
    except NoSuchElementException:
        pass

    # get price find_element_by_class_name
    try:
        d['price'] = DRIVER.find_elements_by_class_name('price-discount__price')
    except NoSuchElementException:
        d['price'] = -1
    
    # get link from .get_attribute()
    try:
        product_link     = product.get_attribute('href')
        d['product_url'] = #YOUR CODE HERE CLEAN LINK WITH STRING MANUPULATION
    except NoSuchElementException:
        pass
    
    # get thumbnail by class_name and Tag name and get_attribute()
    try:
        thumbnail = #YOUR CODE HERE
        d['image'] = #YOUR CODE HERE
    except NoSuchElementException:
        pass
    
    return d


In [ ]:
######################
### START SCRAPING ###
######################

num_max_page = 2
main_cat_url = MAIN_CATEGORIES[-1]['URL']

start_driver(force_restart=True)

#CODE TO GET DATA

close_driver()

#SAVE DATA TO CSV FILE
df = pd.DataFrame(data=prod_data, columns=prod_data[0].keys())
df.to_csv('tiki_products.csv')


Found 64 products
[{'name': 'AdSách Nói: Cà phê cùng Tony', 'price': '10.000 ₫', 'product_url': 'https://tiki.vnhttps://tka.tiki.vn/pixel?data=djAwMcC4akKaiUoHMy4XuWhY57qpr06i_RDiPC7ejP9EONcDmxcK7XAACJjEl0g1E8FucvDAMz8aY8uZGLA1cvTox5fFXq6FgeWAtSLM9Z53xrlMsViiMi77hm2pRe_ceyXkSoTaiphKmRtsw9573Bs4Yq2vwBKTrKvSJt7u-0tZ-Moe4KY2ylMJ-Wi7H_5OBAifp7lt4bJ6z0A-1QnLvNh4aMJ69eatEgkFM0KyjfSMParn600-M8sQ-sIbgvgOQQzUZT-9sBQUKOcTuJJCVBIYhf9KY3USwjzuGUsw579MmXSIVobC81uMUlYxWyuieKK7cbYSe5QTOt_XauYx03cFvd8TKLkj10zo1vqYJiFasknlYENzvQdvSwBIInOGnw4IxQg7NiNRbpi2pACvRA34ZOBAxm00g4LR-n1xG78CfhI7EReMb6ZCo7lfnC0GgMJ4szbJSbqz45-jE9c1NgUtU3CZqIznuBgToiftFr2pXuEtvNddvK0qSMuVxFzwhC--ystH2wUNNi1l-_A1kEY7mIfi4bbpuVEwRJEgijwkHaM_C2S0N67eo0L21Ay1J13NTNV6Ze-4-K4CIoW24M5ra3sKdWuGADNxKDnObXPAiGe1aq7-Atybhv61H3TS4ythFWzAW0mrj45OJxmpIkFjMooMVM3E7HM_hkWXXdEiDVoAaf37Ro3m0NM4hXerddwgN_OEhcUkNrtTGEhkv7LHAIGyfzsyuVAluxkzQovTITn3C5tu_fM0J1BRzMivTcDFH73Kvjqm_2HV6KCBcxQE7hyZyyEDDMHTGLUKbRBgQkZDIqLaIirG5yxdx3eFHByU9AgM4e5_3WC7ZKiPonpl9

In [ ]:
      name = product_dict.get('name')
      price = product_dict.get('price')
      product_url = product_dict.get('product_url')
      image = product_dict.get('image')

In [ ]:
# brief overview of scraped data
df.head()

,name,price,product_url,image
0,AdSách Nói: Cà phê cùng Tony,10.000 ₫,https://tiki.vnhttps://tka.tiki.vn/pixel?data=...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,Phiếu Quà Tặng Tiki 50.000đ,50.000 ₫,https://tiki.vnhttps://tiki.vn/phieu-qua-tang-...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...
2,Phiếu Quà Tặng Tiki 100.000đ,100.000 ₫,https://tiki.vnhttps://tiki.vn/phieu-qua-tang-...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...
3,,,https://tiki.vnhttps://tiki.vn/phieu-qua-tang-...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...
4,AdSinh Trắc Vân Tay Gen Talents – Combo 25 Bài...,26.000.000 ₫,https://tiki.vnhttps://tka.tiki.vn/pixel?data=...,https://salt.tikicdn.com/cache/280x280/ts/prod...
